# Comparison

- 在本 demo 之中，我们将完成不同 extractor 和 matcher 的匹配与效果的比对，主要是对于重建的影响，进而影响定位的效果。

- 在本次实验中，一共有多种 extractor 和 matcher，我们选择其中的几种进行比较探索

In [ ]:
%load_ext autoreload
%autoreload 2
import tqdm, tqdm.notebook

tqdm.tqdm = tqdm.notebook.tqdm  # notebook-friendly progress bars
from pathlib import Path
import numpy as np

from hloc import (
    extract_features,
    match_features,
    reconstruction,
    visualization,
    pairs_from_exhaustive,
)
from hloc.visualization import plot_images, read_image
from hloc.utils import viz_3d

### Set Paths

In [ ]:
images = Path("project/images/MainLibrary/ReconstructionDataset")

extracter = "r2d2"
matcher = "NN-superpoint"

name = extracter + "_" + matcher

output = Path("project/Comparison/" + name)

print(output)

sfm_pairs = output / "pairs-sfm.txt"
loc_pairs = output / "pairs-loc.txt"
sfm_dir = output / "sfm"
features = output / "features.h5"
matches = output / "matches.h5"

feature_conf = extract_features.confs[extracter]
matcher_conf = match_features.confs[matcher]

In [ ]:
references = [p.relative_to(images).as_posix() for p in (images ).iterdir()]

### Extract features & match

In [ ]:
import torch

torch.cuda.set_device(3)

extract_features.main(
    feature_conf, images, image_list=references, feature_path=features
)
pairs_from_exhaustive.main(sfm_pairs, image_list=references)
match_features.main(matcher_conf, sfm_pairs, features=features, matches=matches)

In [ ]:
# extract_features.main(
#     feature_conf2, images, image_list=references, feature_path=features_2
# )
# pairs_from_exhaustive.main(sfm_pairs_2, image_list=references)
# match_features.main(matcher_conf2, sfm_pairs_2, features=features_2, matches=matches_2)

### Reconstruction

In [ ]:
name_1 = "r2d2_NN-superpoint"
path_1 = Path("project/Comparison/" + name_1)
sfm_pairs_1 = path_1 / "pairs-sfm.txt"
sfm_dir_1 = path_1 / "sfm"
features_1 = path_1 / "features.h5"
matches_1 = path_1 / "matches.h5"


model_1 = reconstruction.main(
    sfm_dir_1, images, sfm_pairs_1, features_1, matches_1, image_list=references
)

fig_1 = viz_3d.init_figure()
viz_3d.plot_reconstruction(
    fig_1, model_1, color="rgba(255,0,0,0.5)", name="mapping", points_rgb=True
)


In [ ]:
torch.cuda.set_device(3)

name_2 = ""
path_2 = Path("project/Comparison/" + name_1)
sfm_pairs_2 = path_2 / "pairs-sfm.txt"
sfm_dir_2 = path_2 / "sfm"
features_2 = path_2 / "features.h5"
matches_2 = path_2 / "matches.h5"

model2 = reconstruction.main(
    sfm_dir_2, images, sfm_pairs_2, features_2, matches_2, image_list=references
)

fig_2 = viz_3d.init_figure()
viz_3d.plot_reconstruction(
    fig_2, model2, color="rgba(255,0,0,0.5)", name="mapping", points_rgb=True
)

### Do comparison

In [ ]:
fig_1.show()
fig_2.show()

In [ ]:
fig_1.show()